## Import Packages, Dataset

In [2]:
import pandas as pd
from datetime import datetime
import os,sys
import numpy as np
import matplotlib.pyplot as plt
import glob


%pylab inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Populating the interactive namespace from numpy and matplotlib


C:\Users\daphn\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
os.chdir(r'C:\Users\daphn\Documents\Data_Mining')

In [3]:
data = pd.read_csv("dataset_mood_smartphone.csv", index_col ="id")  #data.head()data.tail() #data.info()

## Preprocessing

### Split time & date, make original timedate into a datetime type, and remove NaN

In [4]:
new = data.dropna()
new.rename(columns={'time':'datetime'},inplace=True)
new['date']=pd.to_datetime(new['datetime'].astype(str).str[:-13],format='%Y-%m-%d')
new = new.drop(["Unnamed: 0",'datetime'], axis=1)

C:\Users\daphn\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
C:\Users\daphn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Explore the Data

In [5]:
#new.isnull().values.any()
print('\n1. There are',len(data[data.isnull().any(axis=1)]), 'values missing. There are',len(new),'complete entries.')
## VARIABLES
var=list(new.variable.unique())
print('2. There are', len(var),'variables.')
print('->',var)
## PARTICIPANTS
ids = new.groupby(['id'])
print('3. There are',len(ids),'participants.')

##TIMES
dates = new.groupby(['date'])
print('4. There are', len(dates),'days. The earliest was ',new['date'].min(),'and the latest',new['date'].max(),'.')

##Entries
gps = new.groupby(['id','variable','date']).size()
print('5. None null variables had at least',gps.min(),'entries per day and maximum',gps.max(),'entries per day.')
sim = new.groupby(['id','variable','date']).size()
print('\tThere are maximum',sim.max(),'simultaneous entries.')


1. There are 202 values missing. There are 376710 complete entries.
2. There are 19 variables.
-> ['mood', 'circumplex.arousal', 'circumplex.valence', 'activity', 'screen', 'call', 'sms', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']
3. There are 27 participants.
4. There are 113 days. The earliest was  2014-02-17 00:00:00 and the latest 2014-06-09 00:00:00 .
5. None null variables had at least 1 entries per day and maximum 377 entries per day.
	There are maximum 377 simultaneous entries.


In [6]:
print('8. These variables have values under 0:\n',list(new.loc[new['value']<0].variable.unique()))

8. These variables have values under 0:
 ['circumplex.arousal', 'circumplex.valence', 'appCat.builtin', 'appCat.entertainment']


## Action needed

### Need to do something for appCat.builtin and appCat. entertainment values that are under 0

### Have summaries of the data

In [7]:
print('In fact here is a summary of the variables.')
for i in var:
    print(i)
    print(new[new['variable']==i]['value'].describe())
    print(new[new['variable']==i]['value'].isnull().values.any())

In fact here is a summary of the variables.
mood
count    5641.000000
mean        6.992555
std         1.032769
min         1.000000
25%         7.000000
50%         7.000000
75%         8.000000
max        10.000000
Name: value, dtype: float64
False
circumplex.arousal
count    5597.000000
mean       -0.098624
std         1.051868
min        -2.000000
25%        -1.000000
50%         0.000000
75%         1.000000
max         2.000000
Name: value, dtype: float64
False
circumplex.valence
count    5487.000000
mean        0.687808
std         0.671298
min        -2.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: value, dtype: float64
False
activity
count    22965.000000
mean         0.115958
std          0.186946
min          0.000000
25%          0.000000
50%          0.021739
75%          0.158333
max          1.000000
Name: value, dtype: float64
False
screen
count    96578.000000
mean        75.335206
std        253.822497
min          0.

## Extracting features

In [8]:
moods= new.loc[(new['variable'] == 'mood')]
active = new.loc[(new.variable == 'activity')]
screens = new.loc[(new.variable == 'screen')]
valence = new.loc[(new.variable == 'circumplex.valence')]
arousal = new.loc[(new.variable == 'circumplex.arousal')]
calls = new.loc[(new.variable == 'call')]
smss= new.loc[(new.variable == 'sms')]
apps = new.loc[(new.variable == 'appCat.builtin')]
coms = new.loc[(new.variable == 'appCat.communication')]
entert = new.loc[(new.variable == 'appCat.entertainment')]
finances = new.loc[(new.variable == 'appCat.finance')]
games = new.loc[(new.variable == 'appCat.game')]
offices = new.loc[(new.variable == 'appCat.office')]
others = new.loc[(new.variable == 'appCat.other')]
social = new.loc[(new.variable == 'appCat.social')]
travels = new.loc[(new.variable == 'appCat.travel')]
unknown = new.loc[(new.variable == 'appCat.unknown')]
utilities = new.loc[(new.variable == 'appCat.utilities')]
weather = new.loc[(new.variable == 'appCat.weather')]
# new.loc[(new.variable == '')]

### COMPUTING ALL FEATURES

#### 1. MOOD (score)

In [9]:
# AVERAGE MOOD per day
mood_mean=moods.groupby(["date","id"]).value.mean().to_frame()

#MOOD DISTANCE FROM AVERAGE

#### 2. 3. CIRCUMPLEX (score)

In [10]:
# Mean per day
valence_mean = valence.groupby(["date", "id"]).value.mean().to_frame()
arousal_mean = arousal.groupby(["date", "id"]).value.mean().to_frame()

# Standard Deviation per day
valence_std = valence.groupby(["date", "id"]).value.std().to_frame()
arousal_std = arousal.groupby(["date", "id"]).value.std().to_frame()

# New circumplex score
circumplex_mean = (valence_mean.value*(arousal_mean.value+4)).to_frame()

#### 4. SCREEN FEATURES (seconds)

In [11]:
# Sum per day
screen_sum= screens.groupby(["date", "id"]).value.sum().to_frame()

# Standard Deviation per day
screen_std = screens.groupby(["date", "id"]).value.std().to_frame()

#Sum of OFF-TIME per day
disconnected = (86400-screen_sum.value).to_frame()

#### 5. ACTIVITY ( score 0 to 1)

In [12]:
# Mean per day
activity_mean = active.groupby(["date", "id"]).value.mean().to_frame()

#Standard Deviation per day
activity_std = active.groupby(["date", "id"]).value.std().to_frame()

#### 6. 7. NUMBER OF CALLS & TEXTS

In [13]:
# Sum per day
calls_sum = calls.groupby(["date", "id"]).value.sum().to_frame()

# Sum per day
smss_sum = smss.groupby(["date", "id"]).value.sum().to_frame()

#### 8. ALL APPS (seconds)

In [14]:
# Sum per day
apps_sum = apps.groupby(["date", "id"]).value.sum().to_frame()

# Standard Deviation per day
apps_std = apps.groupby(["date", "id"]).value.std().to_frame()

#### 9. 19. APPS (seconds)

In [15]:
##COMMUNICATION
# Sum per day
coms_sum = coms.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
coms_std = coms.groupby(["date", "id"]).value.std().to_frame()

##ENTERTAINMENT
# Sum per day
entert_sum = entert.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
entert_std = entert.groupby(["date", "id"]).value.std().to_frame()

##FINANCES
# Sum per day
finances_sum = finances.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
finances_std = finances.groupby(["date", "id"]).value.std().to_frame()

##GAMES
# Sum per day
games_sum = games.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
games_std = games.groupby(["date", "id"]).value.std().to_frame()

##OFFICES
# Sum per day
offices_sum = offices.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
offices_std = offices.groupby(["date", "id"]).value.std().to_frame()

##OTHERS
# Sum per day
others_sum = others.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
others_std = others.groupby(["date", "id"]).value.std().to_frame()

##SOCIAL APPS
# Sum per day
social_sum = social.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
social_std = social.groupby(["date", "id"]).value.std().to_frame()

##TRAVEL APPS
# Sum per day
travels_sum = travels.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
travels_std = travels.groupby(["date", "id"]).value.std().to_frame()

##UNKNOWN
# Sum per day
unknown_sum = unknown.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
unknown_std = unknown.groupby(["date", "id"]).value.std().to_frame()

##UTILITIES
# Sum per day
utilities_sum = utilities.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
utilities_std = utilities.groupby(["date", "id"]).value.std().to_frame()

##WEATHER
# Sum per day
weather_sum = weather.groupby(["date", "id"]).value.sum().to_frame()
# Standard Deviation per day
weather_std = weather.groupby(["date", "id"]).value.std().to_frame()

### Merging features based on Date and ID groups

#### NaN if absent information instead of deleting info.

In [16]:
all_data = mood_mean.merge(activity_mean,\
on=['date','id'],how='left').merge(activity_std, on=['date','id'],how='left')\
.merge(screen_sum, on=['date','id'],how='left').merge(screen_std, on=['date','id'],how='left')\
.merge(disconnected, on=['date','id'],how='left')\
.merge(valence_mean, on=['date','id'],how='outer').merge(valence_std, on=['date','id'],how='outer')\
.merge(arousal_mean, on=['date','id'],how='outer').merge(arousal_std, on=['date','id'],how='outer')\
.merge(circumplex_mean, on=['date','id'],how='outer').merge(calls_sum, on=['date','id'],how='left')\
.merge(smss_sum, on=['date','id'],how='left').merge(apps_sum, on=['date','id'],how='left')\
.merge(apps_std, on=['date','id'],how='left').merge(coms_sum, on=['date','id'],how='left')\
.merge(coms_std, on=['date','id'],how='left').merge(entert_sum, on=['date','id'],how='left')\
.merge(entert_std, on=['date','id'],how='left').merge(finances_sum, on=['date','id'],how='left')\
.merge(finances_std, on=['date','id'],how='left').merge(games_sum, on=['date','id'],how='left')\
.merge(games_std, on=['date','id'],how='left').merge(offices_sum, on=['date','id'],how='left')\
.merge(offices_std, on=['date','id'],how='left').merge(others_sum, on=['date','id'],how='left')\
.merge(others_std, on=['date','id'],how='left').merge(social_sum, on=['date','id'],how='left')\
.merge(social_std, on=['date','id'],how='left').merge(travels_sum, on=['date','id'],how='left')\
.merge(travels_std, on=['date','id'],how='left').merge(unknown_sum, on=['date','id'],how='left')\
.merge(unknown_std, on=['date','id'],how='left').merge(utilities_sum, on=['date','id'],how='left')\
.merge(utilities_std, on=['date','id'],how='left').merge(weather_sum, on=['date','id'],how='left')\
.merge(weather_std, on=['date','id'],how='left')
all_data.columns= ['Mood','Active_mean','Active_std','ScreenTime','Screentime_std','OffTime'\
                     ,'Circ_val','Circ_val_std','Circ_aro','Circ_aro_std','Circumplex','Calls'\
                     ,'SMS','All_apps','All_aps_std','Com_app','Com_app_std','Entertain_app',\
                     'Entertain_app_std','Financ_app','Financ_app_std','Games_app','Games_app_std',\
                     'Offic_app','Offic_app_std','Other_app','Other_app_std','Social_app','Social_app_std',\
                     'Travel_app','Travel_app_std','Unknown_app','Unknown_app_std',\
                     'Util_app','Util_app_std','Weather_app','Weather_app_std']


## FINAL DATA

### Add a week-day feature

In [17]:
all_data = all_data.reset_index()
all_data['days']=all_data['date'].dt.day_name()
all_data.tail(2)

,date,id,Mood,Active_mean,Active_std,ScreenTime,Screentime_std,OffTime,Circ_val,Circ_val_std,Circ_aro,Circ_aro_std,Circumplex,Calls,SMS,All_apps,All_aps_std,Com_app,Com_app_std,Entertain_app,Entertain_app_std,Financ_app,Financ_app_std,Games_app,Games_app_std,Offic_app,Offic_app_std,Other_app,Other_app_std,Social_app,Social_app_std,Travel_app,Travel_app_std,Unknown_app,Unknown_app_std,Util_app,Util_app_std,Weather_app,Weather_app_std,days
1266,2014-06-07,AS14.24,8.0,0.081893,0.210804,5530.377999,507.329479,80869.622001,NaN,NaN,-1.0,NaN,NaN,NaN,2.0,5027.498,507.669997,735.965,43.456539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.214,NaN,106.85,8.693171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saturday
1267,2014-06-08,AS14.24,6.5,0.079510,0.085201,1131.197999,243.226549,85268.802001,1.0,0.0,-1.5,0.707107,2.5,NaN,NaN,351.389,19.665379,375.515,261.227165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.069,0.019092,190.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday


### There are now 40 features

In [18]:
size(all_data.tail(1))

40

### Create a csv file per participant

In [19]:
for i, g in all_data.groupby('id'):
    g.to_csv('{}.csv'.format(i), header=True, index_label='Index')

### Calculate the distance to the Average mood, per participant

In [20]:
temp=[]
participants = glob.glob('AS*.csv')
for i in participants:
    thedata = pd.read_csv(i, index_col ="id")
    mean = thedata.groupby('id').Mood.mean()
    thedata['Mood_distance']=thedata['Mood']-mean
    thedata.drop(columns='Index', axis=1,inplace=True)
    thedata.to_csv(i)
    temp.append(thedata)

### Aggregate all participants in ONE FINAL DATAFRAME

In [23]:
final_data = pd.concat(temp, axis=0, ignore_index=False)
final_data.head(10)


,date,Mood,Active_mean,Active_std,ScreenTime,Screentime_std,OffTime,Circ_val,Circ_val_std,Circ_aro,Circ_aro_std,Circumplex,Calls,SMS,All_apps,All_aps_std,Com_app,Com_app_std,Entertain_app,Entertain_app_std,Financ_app,Financ_app_std,Games_app,Games_app_std,Offic_app,Offic_app_std,Other_app,Other_app_std,Social_app,Social_app_std,Travel_app,Travel_app_std,Unknown_app,Unknown_app_std,Util_app,Util_app_std,Weather_app,Weather_app_std,days,Mood_distance
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AS14.01,2014-02-26,6.250000,NaN,NaN,NaN,NaN,NaN,0.750000,0.500000,-0.25,0.957427,2.812500,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,-0.812411
AS14.01,2014-02-27,6.333333,NaN,NaN,NaN,NaN,NaN,0.333333,0.577350,0.00,1.732051,1.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,-0.729078
AS14.01,2014-03-21,6.200000,0.134050,0.221478,17978.907000,317.499423,68421.093000,0.200000,0.447214,0.20,0.836660,0.840000,6.0,NaN,3139.218,46.233728,6280.890,77.186542,1007.456,73.739846,49.544,3.919842,NaN,NaN,172.206,49.663571,239.751,35.063968,4508.500,92.895360,915.445,122.440366,NaN,NaN,598.754,52.370984,NaN,NaN,Friday,-0.862411
AS14.01,2014-03-22,6.400000,0.236880,0.166143,6142.161000,70.027748,80257.839000,0.500000,1.000000,0.60,0.894427,2.300000,3.0,1.0,731.429,26.029847,4962.918,71.163849,93.324,60.319037,21.076,0.999001,NaN,NaN,NaN,NaN,98.143,10.635952,439.632,37.593916,37.305,NaN,NaN,NaN,117.621,22.125264,NaN,NaN,Saturday,-0.662411
AS14.01,2014-03-23,6.800000,0.142741,0.114999,6773.832001,107.478310,79626.167999,0.800000,0.447214,0.20,0.836660,3.360000,NaN,NaN,1286.246,50.533579,5237.319,92.262747,94.346,23.829479,43.403,8.657808,NaN,NaN,NaN,NaN,72.823,10.159485,900.839,61.168396,NaN,NaN,NaN,NaN,30.086,NaN,30.386,NaN,Sunday,-0.262411
AS14.01,2014-03-24,6.000000,0.078961,0.104793,15047.351001,154.984715,71352.648999,0.000000,0.707107,0.80,1.303840,0.000000,10.0,NaN,866.956,5.995683,9270.629,99.211132,976.971,217.434086,34.106,1.639527,NaN,NaN,3.010,NaN,66.558,1.711579,3223.626,114.370936,419.805,168.371806,NaN,NaN,178.732,112.149964,NaN,NaN,Monday,-1.062411
AS14.01,2014-03-25,6.750000,0.098374,0.147112,21475.354999,464.817570,64924.645001,0.500000,0.577350,0.50,1.000000,2.250000,NaN,1.0,1032.768,31.823303,10276.751,147.008747,68.206,22.679743,43.054,8.727999,NaN,NaN,NaN,NaN,178.819,37.272075,1919.471,70.766200,NaN,NaN,235.223,52.433283,222.893,12.668620,NaN,NaN,Tuesday,-0.312411
AS14.01,2014-03-26,6.600000,0.101308,0.177598,16423.801000,287.953147,69976.199000,0.600000,0.547723,-0.20,1.303840,2.280000,NaN,NaN,1167.497,17.528001,8988.753,82.950839,910.479,95.450885,52.331,6.202521,NaN,NaN,NaN,NaN,97.498,12.359222,4592.059,215.013647,NaN,NaN,NaN,NaN,33.365,NaN,NaN,NaN,Wednesday,-0.462411
AS14.01,2014-03-27,7.000000,0.159511,0.226285,17442.149999,700.922458,68957.850001,0.800000,0.447214,0.20,1.095445,3.360000,2.0,1.0,1229.327,21.177462,6936.512,88.664531,639.339,71.999388,42.219,7.624575,NaN,NaN,182.451,17.965806,58.532,5.926883,935.381,132.590191,47.314,29.196439,NaN,NaN,179.029,14.280520,NaN,NaN,Thursday,-0.062411
